In [4]:
import import_ipynb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from Analise_dos_dados import dados_60, dados_70, dados_80, dados_90, dados_00, dados_10, dados_juntos
from sklearn.tree import export_graphviz
from IPython.display import Image  
import pydotplus
from io import StringIO

In [3]:
# Define as features e os bancos de dados a serem usados.
todos = [dados_60, dados_70, dados_80, dados_90, dados_00, dados_10]
dados = pd.concat(todos)
features = ['acousticness', 'energy', 'instrumentalness', 'speechiness']
X = dados[features]
y = dados.target
# Separa em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.6, test_size=.4)

## Uso do decision tree
Escolhemos esse modelo como nosso primeiro, para que pudessemos testar sua eficácia e comparar com a random forest, que se trata de uma versão mais complexa desse modelo.

In [5]:
# Uso da biblioteca sklearn para o decisiontree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [6]:
y_pred

array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [7]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6684303350970018


## Precisão do primeiro modelo 
Precisão de aprox 66.84 %, imagina se um 66.84 % das músicas de um artista fossem hits ?? 

In [5]:
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = features,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('musiquinha.png')
Image(graph.create_png())

NameError: name 'clf' is not defined

In [9]:
# Separa mais features para testar o quanto elas podem aumentar a eficácia do classificador
features = ['danceability','energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'chorus_hit']
X = dados[features]
y = dados.target
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.67, test_size=.33)

In [11]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6849981570217472


### Essas variáveis não alteraram significantemente o desempenho do classificador .

## Random forest
Adotamos como segundo modelo a random forest, visto que é intimamente relacionada com o modelo anterior.

In [16]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [18]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7711021009952083


### Precisão de 77.11 % com o uso da random forest

___

## Regressão logística
Como terceiro modelo, foi usado a regressão logística utilizando duas bibliotecas diferentes para comparar os resultados e a metodologia.

In [47]:
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [48]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[4368, 2454],
       [1278, 5465]], dtype=int64)

In [49]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7248802064135643


In [52]:
X_cp = sm.add_constant(X_train)
model = sm.Logit(y_train, X_cp)
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.531396
         Iterations 7


In [53]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 target   No. Observations:                27541
Model:                          Logit   Df Residuals:                    27531
Method:                           MLE   Df Model:                            9
Date:                Thu, 03 Jun 2021   Pseudo R-squ.:                  0.2334
Time:                        08:34:52   Log-Likelihood:                -14635.
converged:                       True   LL-Null:                       -19090.
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                1.5396      0.134     11.515      0.000       1.278       1.802
danceability         3.1102      0.108     28.793      0.000       2.898       3.322
energy              -2.0052      0.121    -16.598      0.000      -2.242      -1.768
loudness             0.1099      0.005     20.581      0.000       0.099       0.120
speechiness         -3.5061      0.195    -17.992      0.000      -3.888      -3.124
acousticness        -1.4107      0.064    -22.129      0.000      -1.536      -1.286
instrumentalness    -3.4337      0.082    -41.984      0.000      -3.594      -3.273
liveness            -0.1508      0.084     -1.786      0.074      -0.316       0.015
valence              0.5076      0.071      7.133      0.000       0.368       0.647
chorus_hit          -0.0027      0.001     -3.553      0.000      -0.004      -0.001
====================================================================================
"""

In [59]:
y_pred = results.predict(sm.add_constant(X_test))

In [64]:
resultado = [1 if y > 0.5 else 0 for y in y_pred]
print("Accuracy:", metrics.accuracy_score(y_test, resultado))

Accuracy: 0.7245853298931073
